In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
from PIL import Image
from IPython.display import HTML
import matplotlib.pyplot as plt

from keras.layers import Dense,Dropout,Input,MaxPooling2D,ZeroPadding2D,Conv2D,Flatten
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam,SGD
from keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator
from keras.utils import to_categorical

In [ ]:
labels=pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
labels.head()

In [ ]:
labels.shape

In [ ]:
labels.info()

In [ ]:
labels.describe()

In [ ]:
unique_breeds = labels['breed'].unique()
len(unique_breeds)

In [ ]:
# path of the dogs images
img_file='/kaggle/input/dog-breed-identification/train/'
labels=labels.assign(img_path=lambda x: img_file + x['id'] +'.jpg')
labels.head()

In [ ]:
a=labels['breed'].value_counts()
pd.DataFrame(a).head()

so most popular breed in dataset is scottish deerhound with 126 images.

In [ ]:
ax=pd.value_counts(labels['breed'],ascending=True).plot(kind='barh',fontsize="40",title="Class Distribution",figsize=(50,100))
ax.set(xlabel="Images per class", ylabel="Classes")
ax.xaxis.label.set_size(40)
ax.yaxis.label.set_size(40)
ax.title.set_size(60)
plt.show()

In [ ]:
scottish_deerhound_id = labels.loc[labels.breed == "scottish_deerhound"].iloc[1,0]
Image.open('/kaggle/input/dog-breed-identification/train/'+scottish_deerhound_id+".jpg")

In [ ]:
top_20=list(labels['breed'].value_counts()[:20].index)
top_20

In [ ]:
df=labels[labels['breed'].isin(top_20)]
df.head()

In [ ]:
df.shape

In [ ]:
df['img_path'].values.tolist()

For Machine Learning Operation, we are loading the images and converting it to numpy array of pixel

In [ ]:
img_pixel=np.array([img_to_array(load_img(img,target_size=(299,299))) for img in df['img_path'].values.tolist()])
img_pixel
    

In [ ]:
img_pixel.shape

label encoding of classes(dog breed)

In [ ]:
label1=pd.get_dummies(df['breed'])
label1.head()

In [ ]:
x=img_pixel
y=label1.values#array required in algo

In [ ]:
y

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
train_img_gen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_img_gen=ImageDataGenerator(rescale=1./255)

In [ ]:
train_set=train_img_gen.flow(X_train,y=y_train,batch_size=32)
test_set=test_img_gen.flow(X_test,y=y_test,batch_size=32)

In [ ]:
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),input_shape=(299,299,3),activation='relu',padding='same'))
#model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(20,activation='softmax'))

In [ ]:
model.compile(loss=categorical_crossentropy,optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
batch_size=32
model.fit_generator(
    train_set,
    epochs=2, steps_per_epoch=len(X_train) / batch_size,
    validation_data=(np.array(X_train), np.array(y_train)), validation_steps=len(X_test) / batch_size)